<a href="https://colab.research.google.com/github/dpqhd01/medical_cap/blob/main/Trial_API_%EC%9B%90%EC%9D%B8%ED%83%90%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1단계: 55만 건 '빈 껍데기' 미스터리
시도: "데이터가 더 많으면 해결될까?" 싶어서 55만 건 전체를 가져오려 했습니다.

문제: 55만 건을 요청했는데, eligibilityCriteria 필드가 있는 데이터는 5.5만 건뿐이었습니다. "나머지 50만 건은 '빈 껍데기'인가?"라는 의문이 생겼죠.

v7 스캔 실행 (17분): API 전체를 스캔해서 55만 건의 필드 구조를 '감사(Audit)'했습니다.

원인 발견: 50만 건은 '빈 껍데기'가 아니었습니다! eligibilityCriteria만 없었을 뿐, briefSummary (요약), officialTitle (제목), 심지어 conditions (질병명) 같은 '잭팟' 필드를 가지고 있다는 것을 증명했습니다.

2단계: '슈퍼 텍스트' 빅데이터 구축 (현재)
v8 코드 실행 (17분): 1, 2단계의 모든 문제를 한 번에 해결하는 코드를 실행했습니다.

결과: trials_api_SUPERSET_v1.csv (55.7만 건) 파일 확보에 성공했습니다.

이 파일의 핵심:

빅데이터 확보: 55.7만 건 모두 eligibilityCriteria, summary, title, conditions 등 모든 텍스트를 하나로 합친 **'슈퍼 텍스트'**를 가집니다.

오류 원천 해결: 모든 텍스트에 '텍스트 정규화' (소문자, 공백 제거)를 적용했습니다.

1. API 호출 및 필드 목록 추출

In [8]:
# =========================================================
# 55만 건 전체 데이터 필드(Key) 감사 코드 (v7)
# =========================================================
import requests
import pandas as pd
from tqdm import tqdm
import time
import json

print("⏳ ClinicalTrials.gov API '전체 필드 감사'를 시작합니다... (v7)")

# --- 1. 안전한 필드 추출을 위한 헬퍼 함수 (이전과 동일) ---
def get_nested_value(study_dict, key_path):
    keys = key_path.split('.')
    current_level = study_dict
    for key in keys:
        if isinstance(current_level, dict) and key in current_level:
            current_level = current_level[key]
        else:
            return None
    return current_level

# --- 2. 모든 중첩 키를 찾는 재귀 함수 ---
def find_all_keys(d, key_set, prefix=''):
    """JSON 객체 d의 모든 중첩 키를 찾아 key_set에 추가합니다."""
    if isinstance(d, dict):
        for k, v in d.items():
            key_path = f"{prefix}.{k}" if prefix else k
            key_set.add(key_path)
            # 재귀 호출로 더 깊이 들어감
            find_all_keys(v, key_set, prefix=key_path)
    elif isinstance(d, list):
        # 리스트인 경우, 첫 번째 항목의 구조를 분석 (샘플링)
        if d:
            find_all_keys(d[0], key_set, prefix=f"{prefix}[]")

# --- 3. API 호출 설정 ---
base_url = "https://clinicaltrials.gov/api/v2/studies"
PAGE_SIZE = 1000

# ⚠️ 오류 수정: fields 매개변수를 아예 삭제 (기본값 사용)
# ⚠️ 오류 수정: pageToken을 while 루프에서 동적으로 받도록 수정
params = {
    "pageSize": PAGE_SIZE
}

all_studies_list = []
real_empty_shells = 0

# 모든 고유 필드 이름을 저장할 Set
all_unique_fields = set()
# '빈 껍데기' 연구의 필드 이름을 저장할 Set
empty_shell_fields = set()

next_page_token = None # 시작 토큰은 None
page_count = 0

print("⚠️ 55만 건 전체를 스캔합니다. 1~3시간 이상 소요될 수 있습니다...")

# --- 4. API 루프 (nextPageToken 사용) ---
try:
    with tqdm(desc="전체 데이터 감사 중", unit=" studies") as pbar:
        while True:
            page_count += 1
            if next_page_token:
                params["pageToken"] = next_page_token
            else:
                # 첫 페이지 요청 시 pageToken 삭제
                if "pageToken" in params:
                    del params["pageToken"]

            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            studies_in_page = data.get('studies', [])

            if not studies_in_page:
                print(f"\n데이터가 반환되지 않았습니다. 수집을 중단합니다.")
                break

            for study in studies_in_page:
                # 1. 이 연구의 모든 필드 경로를 수집
                find_all_keys(study, all_unique_fields)

                # 2. '빈 껍데기'인지 확인
                criteria = get_nested_value(study, 'protocolSection.eligibilityModule.eligibilityCriteria')
                summary = get_nested_value(study, 'protocolSection.descriptionModule.briefSummary')
                title = get_nested_value(study, 'protocolSection.identificationModule.officialTitle')

                # 3개 텍스트 필드가 모두 없는 경우
                if (pd.isna(criteria) or not str(criteria).strip()) and \
                   (pd.isna(summary) or not str(summary).strip()) and \
                   (pd.isna(title) or not str(title).strip()):

                    # '빈 껍데기' 연구의 필드만 따로 수집
                    find_all_keys(study, empty_shell_fields)

            pbar.update(len(studies_in_page))

            # 다음 페이지 토큰 갱신
            next_page_token = data.get('nextPageToken')
            if not next_page_token:
                break # 다음 페이지가 없으면 루프 종료

            time.sleep(0.5) # API 과부하 방지

except Exception as e:
    print(f"\n🔥🔥 {page_count} 페이지 처리 중 오류 발생: {e}")
    # 오류가 발생한 경우 서버가 보낸 메시지를 출력 (디버깅용)
    try:
        print("\n--- 서버 오류 메시지 ---")
        print(response.json())
    except:
        pass

# --- 5. 결과 출력 ---
print("\n🎉🎉🎉 '빅데이터' 전체 필드 감사 완료! 🎉🎉🎉")
print(f"\n--- 1. 55만 건에서 발견된 모든 고유 필드 (일부) ---")
# 필드 목록 정렬
all_fields_list = sorted(list(all_unique_fields))
for field in all_fields_list[:50]: # 너무 많으므로 50개만 출력
    print(field)

print(f"\n... (총 {len(all_fields_list)}개의 고유 필드 경로 발견)")

print(f"\n--- 2. '빈 껍데기' 연구에서만 발견된 고유 필드 (일부) ---")
empty_fields_list = sorted(list(empty_shell_fields))
for field in empty_fields_list[:50]: # 50개만 출력
    print(field)

print(f"\n... (총 {len(empty_fields_list)}개의 고유 필드 경로 발견)")

print("\n\n--- 💡 질환/증상명 관련 추천 필드 ---")
print("목록에서 아래 필드들이 있는지 확인해 보세요:")
print(" - protocolSection.conditionsModule.conditions[]")
print(" - protocolSection.conditionsModule.keywords[]")
print(" - protocolSection.identificationModule.officialTitle")
print(" - protocolSection.descriptionModule.briefSummary")

⏳ ClinicalTrials.gov API '전체 필드 감사'를 시작합니다... (v7)
⚠️ 55만 건 전체를 스캔합니다. 1~3시간 이상 소요될 수 있습니다...


전체 데이터 감사 중: 236000 studies [07:18, 538.26 studies/s]


KeyboardInterrupt: 

In [9]:
# =========================================================
# 55만 건 '빅데이터' 최종 수집 코드 (v8)
# =========================================================
import requests
import pandas as pd
from tqdm import tqdm
import time
import json

print("⏳ ClinicalTrials.gov API '빅데이터' 최종 수집을 시작합니다... (v8)")
print("⚠️ 'eligibilityCriteria', 'Summary', 'Title', 'Conditions', 'Keywords'를 병합합니다.")

# --- 1. 안전한 필드 추출을 위한 헬퍼 함수 ---
def get_nested_value(study_dict, key_path):
    """
    study 딕셔너리에서 key_path (예: 'protocolSection.descriptionModule.briefSummary')를
    안전하게 찾아 반환합니다. 없으면 None을 반환합니다.
    """
    keys = key_path.split('.')
    current_level = study_dict
    for key in keys:
        if isinstance(current_level, dict) and key in current_level:
            current_level = current_level[key]
        else:
            return None
    return current_level

# --- 2. API 호출 설정 ---
base_url = "https://clinicaltrials.gov/api/v2/studies"
PAGE_SIZE = 1000

# ⚠️ fields 매개변수를 삭제 (v7 스캔에서 성공한 방식)
# 이렇게 하면 전체 JSON을 가져오지만, 55만 건을 17분 만에 처리한 것이 증명됨.
params = {
    "pageSize": PAGE_SIZE
}

# 최종 텍스트 데이터를 저장할 리스트
all_super_texts = []
real_empty_shells = 0 # 모든 텍스트 필드가 없는 경우

next_page_token = None # 시작 토큰은 None
page_count = 0

print("⚠️ 55만 건 전체를 스캔합니다. 약 20~30분 소요될 수 있습니다...")

# --- 3. API 루프 (nextPageToken 사용) ---
try:
    with tqdm(desc="빅데이터 수집 중", unit=" studies") as pbar:
        while True:
            page_count += 1
            if next_page_token:
                params["pageToken"] = next_page_token
            else:
                if "pageToken" in params:
                    del params["pageToken"]

            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            studies_in_page = data.get('studies', [])

            if not studies_in_page:
                print(f"\n데이터가 반환되지 않았습니다. 수집을 중단합니다.")
                break

            for study in studies_in_page:
                # 1. 텍스트 필드 추출 (v7 스캔에서 확인된 필드들)
                criteria = get_nested_value(study, 'protocolSection.eligibilityModule.eligibilityCriteria')
                summary = get_nested_value(study, 'protocolSection.descriptionModule.briefSummary')
                title = get_nested_value(study, 'protocolSection.identificationModule.officialTitle')

                # 2. '잭팟' 필드 추출 (리스트 형태이므로 join 필요)
                conditions_list = get_nested_value(study, 'protocolSection.conditionsModule.conditions')
                keywords_list = get_nested_value(study, 'protocolSection.conditionsModule.keywords')

                conditions_text = " ".join(conditions_list or [])
                keywords_text = " ".join(keywords_list or [])

                # 3. '슈퍼 텍스트' 생성 로직
                # 3-1. 기본 텍스트 (Criteria 우선, 없으면 Summary + Title)
                base_text = ""
                if criteria and str(criteria).strip():
                    base_text = criteria
                else:
                    base_text = (summary or "") + " " + (title or "")

                # 3-2. 모든 텍스트 병합
                final_combined_text = (
                    (base_text or "") + " " +
                    (conditions_text or "") + " " +
                    (keywords_text or "")
                ).strip() # 앞뒤 공백 제거

                # 4. 유효한 텍스트만 리스트에 추가
                if final_combined_text:
                    all_super_texts.append({'super_text': final_combined_text})
                else:
                    real_empty_shells += 1 # 모든 필드가 비어있는 '진짜 빈 껍데기'

            pbar.update(len(studies_in_page))

            # 다음 페이지 토큰 갱신
            next_page_token = data.get('nextPageToken')
            if not next_page_token:
                break # 다음 페이지가 없으면 루프 종료

            time.sleep(0.5) # API 과부하 방지

except Exception as e:
    print(f"\n🔥🔥 {page_count} 페이지 처리 중 오류 발생: {e}")
    try:
        print("\n--- 서버 오류 메시지 ---")
        print(response.json())
    except:
        pass

# --- 4. 결과 저장 ---
print("\n🎉🎉🎉 '빅데이터' 최종 수집 완료! 🎉🎉🎉")
print(f"총 {len(all_super_texts)}건의 유효한 '슈퍼 텍스트' 데이터를 확보했습니다.")
print(f"(모든 텍스트 필드가 없는 '진짜 빈 껍데기' {real_empty_shells}건 제외)")

# 결과를 새로운 CSV 파일로 저장
df_big_data = pd.DataFrame(all_super_texts)
df_big_data.to_csv("trials_api_SUPERSET_v1.csv", index=False, encoding='utf-8-sig')
print("✅ 'trials_api_SUPERSET_v1.csv' 파일 저장 완료!")

⏳ ClinicalTrials.gov API '빅데이터' 최종 수집을 시작합니다... (v8)
⚠️ 'eligibilityCriteria', 'Summary', 'Title', 'Conditions', 'Keywords'를 병합합니다.
⚠️ 55만 건 전체를 스캔합니다. 약 20~30분 소요될 수 있습니다...


빅데이터 수집 중: 557256 studies [16:29, 563.08 studies/s]



🎉🎉🎉 '빅데이터' 최종 수집 완료! 🎉🎉🎉
총 557256건의 유효한 '슈퍼 텍스트' 데이터를 확보했습니다.
(모든 텍스트 필드가 없는 '진짜 빈 껍데기' 0건 제외)
✅ 'trials_api_SUPERSET_v1.csv' 파일 저장 완료!


In [14]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

# 파일을 저장할 경로 (예: Drive의 최상위 폴더)
DRIVE_SAVE_PATH = '/content/drive/MyDrive/'

# 만약 'Colab Notebooks' 폴더 안에 저장하고 싶다면 아래 주석을 해제하세요.
# DRIVE_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/'
# os.makedirs(DRIVE_SAVE_PATH, exist_ok=True) # 폴더가 없으면 생성

print(f"✅ Google Drive가 마운트되었습니다. 파일 저장 경로: {DRIVE_SAVE_PATH}")

Mounted at /content/drive
✅ Google Drive가 마운트되었습니다. 파일 저장 경로: /content/drive/MyDrive/


In [15]:
import shutil
import os # os.path.join을 위해 import

# 원본 파일 경로 (코랩 로컬)
SOURCE_FILE_PATH = 'trials_api_SUPERSET_v1.csv'

# ⚠️ 1단계에서 설정된 DRIVE_SAVE_PATH 변수를 사용하여 대상 경로를 올바르게 설정
DESTINATION_PATH = os.path.join(DRIVE_SAVE_PATH, SOURCE_FILE_PATH)

try:
    print(f"⏳ '{SOURCE_FILE_PATH}' 파일을 Google Drive로 복사합니다...")
    print(f"대상 경로: {DESTINATION_PATH}") # 👈 이제 /content/drive/MyDrive/... 로 표시되어야 합니다.

    shutil.copy(SOURCE_FILE_PATH, DESTINATION_PATH)

    print("\n🎉🎉🎉 파일 복사 완료! 🎉🎉🎉")
    print("이제 (동기화 지연을 감안하여 5~10분 후) Google Drive에서 파일을 확인하실 수 있습니다.")

except FileNotFoundError:
    print(f"🔥🔥 오류: 원본 파일 '{SOURCE_FILE_PATH}'을(를) 찾을 수 없습니다.")
except Exception as e:
    print(f"🔥🔥 파일 복사 중 오류 발생: {e}")

⏳ 'trials_api_SUPERSET_v1.csv' 파일을 Google Drive로 복사합니다...
대상 경로: /content/drive/MyDrive/trials_api_SUPERSET_v1.csv

🎉🎉🎉 파일 복사 완료! 🎉🎉🎉
이제 (동기화 지연을 감안하여 5~10분 후) Google Drive에서 파일을 확인하실 수 있습니다.


In [ ]:
# =========================================================
# Code Block 1: '빅데이터' 임베딩 생성 (Step 2)
# ✨ Google Drive 자동 저장 기능 추가
# =========================================================
!pip install sentence-transformers tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch
import os
import shutil # 👈 (1) 파일 복사 라이브러리
from google.colab import drive # 👈 (2) Google Drive 라이브러리

# --- (3) Google Drive 마운트 및 저장 경로 설정 ---
# 이 코드를 실행하면 Drive 마운트 인증 팝업이 뜹니다.
try:
    drive.mount('/content/drive', force_remount=True)
    DRIVE_SAVE_PATH = '/content/drive/MyDrive/' # 👈 저장할 Drive 경로 (내 드라이브)
    os.makedirs(DRIVE_SAVE_PATH, exist_ok=True)
    print(f"✅ Google Drive 마운트 완료. 저장 경로: {DRIVE_SAVE_PATH}")
except Exception as e:
    print(f"🔥🔥 Google Drive 마운트 실패: {e}")
    DRIVE_SAVE_PATH = None # Drive 저장 실패 시 None으로 설정

# 텍스트 정규화 함수 (매칭 오류 해결 핵심)
def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip().lower().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')


print("⏳ 임베딩 모델 로드 중...")
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
print("✅ 임베딩 모델 로드 완료!")

# --- 1. '빅데이터' (Superset) 임베딩 ---
print("\n⏳ 'trials_api_SUPERSET_v1.csv' 파일을 임베딩합니다... (시간 소요)")
# ⚠️ 'trials_api_SUPERSET_v1.csv' 파일이 로컬에 있어야 합니다.
df_superset = pd.read_csv("trials_api_SUPERSET_v1.csv")

# ⚠️ 텍스트 정규화 적용 (매우 중요!)
df_superset['super_text_normalized'] = df_superset['super_text'].apply(normalize_text)
texts_superset = df_superset['super_text_normalized'].tolist()

# 모델 인코딩 (GPU 사용 권장)
embeddings_superset = model.encode(texts_superset, show_progress_bar=True, convert_to_numpy=True)

# 결과 저장
embedding_df_superset = pd.DataFrame(embeddings_superset)
text_df_superset = pd.DataFrame(texts_superset, columns=['original_text'])
final_df_superset = pd.concat([text_df_superset, embedding_df_superset], axis=1)

# 💥 새로운 빅데이터 임베딩 파일 (로컬 저장)
LOCAL_SUPERSET_FILE = "trial_embeddings_SUPERSET.csv"
final_df_superset.to_csv(LOCAL_SUPERSET_FILE, index=False)
print(f"✅ '{LOCAL_SUPERSET_FILE}' 파일 로컬 저장 완료!")

# --- (4) 생성된 파일 자동 백업 (Superset) ---
if DRIVE_SAVE_PATH:
    try:
        print(f"⏳ '{LOCAL_SUPERSET_FILE}' 파일을 Google Drive로 백업합니다...")
        shutil.copy(LOCAL_SUPERSET_FILE, os.path.join(DRIVE_SAVE_PATH, LOCAL_SUPERSET_FILE))
        print(f"✅ '{LOCAL_SUPERSET_FILE}' 파일 Google Drive 백업 완료!")
    except Exception as e:
        print(f"🔥🔥 Drive 백업 실패: {e}")


# --- 2. Orphanet (HPO) 데이터 임베딩 ---
print("\n⏳ 'out4.csv' 파일을 임베딩합니다... (텍스트 정규화 적용)")
# ⚠️ 'out4.csv' 파일이 로컬에 있어야 합니다.
df_hpo = pd.read_csv("/content/out4.csv") # 사용자가 사용한 경로 유지
df_hpo['HPOTERM_normalized'] = df_hpo['HPOTERM'].apply(normalize_text)
texts_hpo = df_hpo['HPOTERM_normalized'].tolist()
embeddings_hpo = model.encode(texts_hpo, show_progress_bar=True, convert_to_numpy=True)

embedding_df_hpo = pd.DataFrame(embeddings_hpo)
text_df_hpo = pd.DataFrame(texts_hpo, columns=['original_hpoterm'])
final_df_hpo = pd.concat([text_df_hpo, embedding_df_hpo], axis=1)

# 💥 새로운 HPO 임베딩 파일 (정규화됨)
LOCAL_HPO_FILE = "orphanet_embeddings_NORMALIZED.csv"
final_df_hpo.to_csv(LOCAL_HPO_FILE, index=False)
print(f"✅ '{LOCAL_HPO_FILE}' 파일 로컬 저장 완료!")

# --- (5) 생성된 파일 자동 백업 (HPO) ---
if DRIVE_SAVE_PATH:
    try:
        print(f"⏳ '{LOCAL_HPO_FILE}' 파일을 Google Drive로 백업합니다...")
        shutil.copy(LOCAL_HPO_FILE, os.path.join(DRIVE_SAVE_PATH, LOCAL_HPO_FILE))
        print(f"✅ '{LOCAL_HPO_FILE}' 파일 Google Drive 백업 완료!")
    except Exception as e:
        print(f"🔥🔥 Drive 백업 실패: {e}")

print("\n🎉 모든 임베딩 및 백업 작업이 완료되었습니다. 이제 주무셔도 좋습니다. :)")

Mounted at /content/drive
✅ Google Drive 마운트 완료. 저장 경로: /content/drive/MyDrive/
⏳ 임베딩 모델 로드 중...
✅ 임베딩 모델 로드 완료!

⏳ 'trials_api_SUPERSET_v1.csv' 파일을 임베딩합니다... (시간 소요)


Batches:   0%|          | 0/17415 [00:00<?, ?it/s]